# Сбор данный для обучения модели с сайта auto.ru

In [3]:
#импортирование необходимый библиотек
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import unicodedata
import json

In [3]:
#марки атвомобилей, по которым будет производится сбор данных
brands = {'audi',
 'bmw',
 'honda',
 'infiniti',
 'lexus',
 'mercedes',
 'mitsubishi',
 'nissan',
 'skoda',
 'toyota',
 'volkswagen',
 'volvo'}

In [4]:
def get_car_links(car, year_from = '1900', year_to='2020'):
    """Функция парсит страницу марки и вытаскивает ссылки на объявления.
        На вход получает марку автомобиля и опционально период производства (по умолчанию 1900-2020)
        На выходе список ссылок на все автомобили данной марки с годом производства
        в указанном периоде, размещенные на auto.ru"""
    links =[]
    request = requests.get('https://auto.ru/rossiya/cars/'+car+'/all/?year_from='+year_from+'&year_to='+year_to)
    if request.status_code != 200:
        return "Connection problem"
    request.encoding = 'utf-8'
    car_page = request.text
    car_parse = BeautifulSoup(car_page, 'html.parser')
    car_count = car_parse.find("span", class_="ButtonWithLoader__content",)
    #колличество объеявлений данной марки
    count = int(''.join(unicodedata.normalize("NFKD",car_count.text)[9:-12].split()))
    print('{0} - {1}'.format(car, count))
    p = 1
    while len(links) < count:
        request = requests.get('https://auto.ru/rossiya/cars/'+car+'/all/?page='+str(p)+'&year_from=2007&year_to=2008')
        if request.status_code != 200:
            break
        request.encoding = 'utf-8'
        car_page = request.text
        urls_parse = BeautifulSoup(car_page, 'html.parser')
        time.sleep(2)
        p += 1 #переходим на след. страницу
        cars_urls = urls_parse.find_all("a", class_="Link ListingItemTitle-module__link")
        #Отбираем ссылки ведущие к конкретному предложению и собираем в список
        for item in cars_urls:
            url = item.get('href')
            links.append(url)
    return links #возвращаем ссылки на предложения по модели

In [5]:
def get_car_info(url):
    """Функция собирает информацию по ссылке объявления.
        На вход принимает ссылку на объявление
        На выходе список искомых данных"""
    data =[]
    request = requests.get(url)
    if request.status_code != 200:
        return None    
    request.encoding = 'utf-8'
    car_page = request.text
    car_parse = BeautifulSoup(car_page, 'html.parser')
    #тип кузова
    bodytype_parse = car_parse.find("li", class_='CardInfoRow CardInfoRow_bodytype')
    if bodytype_parse:
        bodytype = bodytype_parse.a.text
        data.append(bodytype)
    else:
        data.append(None)
    #Brand
    brand = url.split('/')[6].upper()
    data.append(brand)    
    #url
    data.append(url)
    #цвет
    color_parse = car_parse.find("li", class_='CardInfoRow CardInfoRow_color')
    if color_parse:
        color = color_parse.a.text
        data.append(color)
    else:
        data.append(None)
    #опции
    options = {}
    addition_items = car_parse.find_all('div', class_='ComplectationGroups__group')
    for option in addition_items:
        name = option.find_all('div')[0]['data-group']
        option_parse = option.ul.find_all('li')
        new = []
        for item in option_parse:
            new.append(item.text)
        options[name] = new
    data.append(options)
    #текст
    text_parse = car_parse.find("div", class_='CardDescription__textInner')
    if text_parse:
        text = text_parse.span.text
        data.append(text)
    else:
        data.append(None)
    #двигатель
    engine_parse = car_parse.find("li", class_='CardInfoRow CardInfoRow_engine')
    if engine_parse:
        engine_parse_clear = unicodedata.normalize("NFKD", engine_parse.find_all('span')[1].text).split('/')
        #объем двигателя
        engine_displacement = float(engine_parse_clear[0].split()[0])
        data.append(engine_displacement)
        #мощьность
        enginepower = int(engine_parse_clear[1].split()[0])
        data.append(enginepower) 
        #топливо
        fuelType = engine_parse_clear[2].strip()
        data.append(fuelType)
    else:
        data.append(None)
        data.append(None)
        data.append(None)        
    #пробег
    mileage_parse = car_parse.find("li", class_='CardInfoRow CardInfoRow_kmAge')
    if mileage_parse:
        mileage = int(''.join(unicodedata.normalize("NFKD", mileage_parse.find_all('span')[1].text)[:-3].split()))
        data.append(mileage) 
    else:
        data.append(None)
    #model
    model = url.split('/')[7]
    data.append(model)
    #name
    name_parse = car_parse.find_all("a", class_='Link Link_color_gray CardBreadcrumbs__itemText')
    if name_parse:
        name = unicodedata.normalize("NFKD", name_parse[4].text)
        data.append(name)
    else:
        data.append(None)
    #год
    year_parse = car_parse.find("li", class_='CardInfoRow CardInfoRow_year')
    if year_parse:
        year = year_parse.a.text
        data.append(year)
    else:
        data.append(None)
    #трансмиссия
    transmission_parse = car_parse.find("li", class_='CardInfoRow CardInfoRow_transmission')
    if transmission_parse:
        transmission = transmission_parse.find_all('span')[1].text
        data.append(transmission)
    else:
        data.append(None)
    #владельцы
    owners_parse = car_parse.find('li', class_='CardInfoRow CardInfoRow_ownersCount')
    if owners_parse:
        owners = owners_parse.find_all('span')[1].text
        data.append(owners)
    else:
        data.append(None)
    #птс
    pts_parse = car_parse.find('li', class_='CardInfoRow CardInfoRow_pts')
    if pts_parse:
        pts = pts_parse.find_all('span')[1].text
        data.append(pts)
    else:
        data.append(None)
    #владение
    owningtime_parse = car_parse.find('li', class_='CardInfoRow CardInfoRow_owningTime')
    if owningtime_parse:
        owningtime = owningtime_parse.find_all('span')[1].text
        data.append(owningtime)
    else:
        data.append(None)
    #привод
    drive_parse = car_parse.find('li', class_='CardInfoRow CardInfoRow_drive')
    if drive_parse:
        drive = drive_parse.find_all('span')[1].text
        data.append(drive)
    else:
        data.append(None)
    #руль
    wheel_parse = car_parse.find('li', class_='CardInfoRow CardInfoRow_wheel')
    if wheel_parse:
        wheel = wheel_parse.find_all('span')[1].text
        data.append(wheel)
    else:
        data.append(None)
    #цена    
    price_parse = car_parse.find("div", class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren PriceUsedOffer-module__price')
    if price_parse: 
        price = int(''.join(unicodedata.normalize("NFKD", price_parse.find_all('span')[1].text)[:-2].split()))
        data.append(price)
    else:
        return None
    return data

In [6]:
#собираем все ссылки на искомые марки в списке brends в единый список car_links
car_links = []
for brand in brands:
    car_links.extend(get_car_links(brand, year_from='1914', year_to='2000'))

bmw - 442


In [49]:
#созраняем данные для дальнейшего использования
with open('data/car_links.json', 'w') as f:
    f.write(json.dumps(car_links))
#загружаем данные
with open('data/car_links.json', 'r') as f:
    car_links = json.load(f)

In [16]:
#собираем данные из сылок в списке car_links, каждые 10000 записей сохраняем в DF и записываем в файл
data = [] 
files = []
file = 0
for item in car_links:
    taken_data = get_car_info(item) 
    time.sleep(2)
    if taken_data:
        data.append(taken_data)
        length = len(data)
    if length%10000 == 0:
        print('data/data{} -  {}%'.format(file, (length/10000)*100)) #периодический вывод информации о протекающем процессе
    if length >= 10000:
        df = pd.DataFrame(data)
        df.to_csv('data/data{}.csv'.format(file), index=False, encoding='utf8') #записываем данные в файл
        files.append('data/data{}.csv'.format(file)) #сохраняем список созданных файлов
        file += 1
        data = [] #обнуляем список
df = pd.DataFrame(data)
files.append('data/data{}.csv'.format(file))
df.to_csv('data/data{}.csv'.format(file), index=False, encoding='utf8') #записываем данные в файл
with open('data/data_files.txt', 'w') as f:
    f.write(json.dumps(files))
print('Parsing complit {} files created'.format(file+1))    

data0 -  1.0%
data1 -  1.0%
data2 -  1.0%
data3 -  1.0%
Parsing complit 5 files created


In [4]:
#собираем данные в единый датафрейм
with open('data/data_files.txt', 'r') as f:
    files = json.load(f)
data_files = [pd.read_csv(fname) for fname in files]
data = pd.concat(data_files)
#сохраняем данные в файл
data.columns = ['bodyType', 'brand', 'car_url', 'color', 'options', 'description', 'engineDisplacement',
                'enginePower', 'fuelType', 'mileage','model_name', 'name', 'productionDate', 'vehicleTransmission',
                'owners', 'pts', 'own', 'drive', 'wheel', 'price']
data.to_csv('data/data.csv', index=False, encoding='utf8')